### Sources
- https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html
- https://towardsdatascience.com/interpretable-machine-learning-models-aef0c7be3fd9

In [3]:
import numpy as np
import shap
import pandas as pd


c:\Users\emilh\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = pd.read_csv('csgo_round_snapshots.csv')
print(data.head(10))
print(data.describe())

   time_left  ct_score  t_score       map  bomb_planted  ct_health  t_health  \
0     175.00       0.0      0.0  de_dust2         False      500.0     500.0   
1     156.03       0.0      0.0  de_dust2         False      500.0     500.0   
2      96.03       0.0      0.0  de_dust2         False      391.0     400.0   
3      76.03       0.0      0.0  de_dust2         False      391.0     400.0   
4     174.97       1.0      0.0  de_dust2         False      500.0     500.0   
5     114.97       1.0      0.0  de_dust2         False      500.0     500.0   
6      94.97       1.0      0.0  de_dust2         False      295.0     111.0   
7      74.97       1.0      0.0  de_dust2         False      295.0     111.0   
8      54.97       1.0      0.0  de_dust2         False      249.0      89.0   
9     174.97       2.0      0.0  de_dust2         False      500.0     500.0   

   ct_armor  t_armor  ct_money  ...  t_grenade_flashbang  \
0       0.0      0.0    4000.0  ...                  0.0   